Загрузка библиотек:

In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torchvision import transforms, datasets

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

Загрузка и разбиение данных:

In [2]:
data = fetch_california_housing()
X, Y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=13)

n_features_reg = X_train.shape[1]

X_train_reg, X_test_reg = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
Y_train_reg, Y_test_reg = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

X_train_reg.shape, X_test_reg.shape, Y_train_reg.shape, Y_test_reg.shape

(torch.Size([15480, 8]),
 torch.Size([5160, 8]),
 torch.Size([15480]),
 torch.Size([5160]))

Регуляризация данных:

In [3]:
mean = X_train_reg.mean(axis=0)
std = X_train_reg.std(axis=0)

X_train_reg = (X_train_reg - mean)/ std
X_test_reg = (X_test_reg - mean)/ std

Класс со своим датасетом:

In [4]:
class LinearRegressionDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

DataLoader:

In [5]:
train_loader = torch.utils.data.DataLoader(dataset = LinearRegressionDataset(X_train, y_train), batch_size=64)
test_loader = torch.utils.data.DataLoader(dataset = LinearRegressionDataset(X_test, y_test), batch_size=64)

Построение модели регрессии:

In [6]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.lin1 = nn.Linear(n_features_reg, 5)
        self.bn = nn.BatchNorm1d(5)
        self.dp = nn.Dropout(0.25)
        self.lin2 = nn.Linear(5, 10)
        self.lin3 = nn.Linear(10, 15)
        self.lin4 = nn.Linear(15,1)

    def forward(self, X_batch):
        layer_out = F.relu(self.lin1(X_batch))
        layer_out = self.bn(layer_out)
        layer_out = self.dp(layer_out)
        layer_out = F.relu(self.lin2(layer_out))
        layer_out = F.relu(self.lin3(layer_out))

        return self.lin4(layer_out).ravel()

regressor = Regressor()
preds = regressor(X_train_reg[:5])
preds

tensor([-0.0813, -0.0751, -0.4667,  0.0268, -0.1443], grad_fn=<ViewBackward0>)

Обучение модели:

In [7]:
def TrainModel(model, loss_func, optimizer, X, Y, epochs=500):
    for i in range(epochs):
        preds = model(X) ## Make Predictions by forward pass through network

        loss = loss_func(preds, Y) ## Calculate Loss

        optimizer.zero_grad() ## Zero weights before calculating gradients
        loss.backward() ## Calculate Gradients
        optimizer.step() ## Update Weights

        if i % 1000 == 0: ## Print Loss every 1000 epochs
            print("Loss : {:.2f}".format(loss))

In [8]:
from torch.optim import SGD, RMSprop, Adam
from sklearn.metrics import r2_score

Тестирование на трех оптимайзерах:

In [9]:
torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 5000
learning_rate = torch.tensor(0.001)

regressor = Regressor()
mse_loss = nn.MSELoss()
optimizer = SGD(params=regressor.parameters(), lr=learning_rate)

TrainModel(regressor, mse_loss, optimizer, X_train_reg, Y_train_reg, epochs=epochs)

Loss : 5.82
Loss : 0.95
Loss : 0.75
Loss : 0.64
Loss : 0.60


In [10]:
test_preds = regressor(X_test_reg)
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy().squeeze(), Y_test_reg.detach().numpy())))

Test  R^2 Score : 0.24


In [11]:
torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 5000
learning_rate = torch.tensor(0.001)

regressor = Regressor()
mse_loss = nn.MSELoss()
optimizer = RMSprop(params=regressor.parameters(), lr=learning_rate)

TrainModel(regressor, mse_loss, optimizer, X_train_reg, Y_train_reg, epochs=epochs)

Loss : 5.82
Loss : 0.41
Loss : 0.37
Loss : 0.37
Loss : 0.37


In [12]:
test_preds = regressor(X_test_reg)
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy().squeeze(), Y_test_reg.detach().numpy())))

Test  R^2 Score : 0.60


In [13]:
torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 5000
learning_rate = torch.tensor(0.001)

regressor = Regressor()
mse_loss = nn.MSELoss()
optimizer = Adam(params=regressor.parameters(), lr=learning_rate)

TrainModel(regressor, mse_loss, optimizer, X_train_reg, Y_train_reg, epochs=epochs)

Loss : 5.82
Loss : 0.47
Loss : 0.42
Loss : 0.40
Loss : 0.39


In [14]:
test_preds = regressor(X_test_reg)
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy().squeeze(), Y_test_reg.detach().numpy())))

Test  R^2 Score : 0.58


Как видно из результатов, лучшим оптимайзером оказался RMS. На втором месте, с чуть похуже метриками Adam, и со значительным отставанием SGD.